In [1]:
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [2]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = 'F:\\Project\\chest_xray\\train'
valid_path = 'F:\\Project\chest_xray\\test'


In [3]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)




In [5]:
print(vgg.output)

KerasTensor(type_spec=TensorSpec(shape=(None, 7, 7, 512), dtype=tf.float32, name=None), name='block5_pool/MaxPool:0', description="created by layer 'block5_pool'")


In [6]:
for i in vgg.layers:
  print(i)

In [7]:
# don't train existing weights
for layer in vgg.layers:
    layer.trainable = False

In [8]:
  # useful for getting number of output classes
folders = glob('F:\\Project\\chest_xray\\train\\*')

In [9]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)

In [10]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [11]:

# view the structure of the model
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [12]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [13]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [14]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('F:\\Project\\chest_xray\\train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 5216 images belonging to 2 classes.


In [15]:
test_set = test_datagen.flow_from_directory('F:\\Project\\chest_xray\\train',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 5216 images belonging to 2 classes.


In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
163/163 [==============================] - 120s 688ms/step - loss: 0.4138 - accuracy: 0.8656 - val_loss: 0.2152 - val_accuracy: 0.9151
Epoch 2/5
163/163 [==============================] - 111s 678ms/step - loss: 0.1090 - accuracy: 0.9599 - val_loss: 0.3639 - val_accuracy: 0.8878
Epoch 3/5
163/163 [==============================] - 110s 674ms/step - loss: 0.1160 - accuracy: 0.9606 - val_loss: 0.2241 - val_accuracy: 0.9279
Epoch 4/5
163/163 [==============================] - 109s 668ms/step - loss: 0.0997 - accuracy: 0.9628 - val_loss: 0.2936 - val_accuracy: 0.9183
Epoch 5/5
163/163 [==============================] - 108s 665ms/step - loss: 0.0655 - accuracy: 0.9788 - val_loss: 0.2539 - val_accuracy: 0.9215


In [ ]:
# save it as a h5 file

import tensorflow as tf

from keras.models import load_model

model.save('F:\Project\final_model.h5')